# Computer Exercises 

In [163]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

from numpy import pi, cos
from sympy import symbols

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Power spectral density, periodogram

$$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$$

In [38]:
A = [250, 34, 8 , 10]

fn = 10
w0 = 2*pi*fn 
w1 = w0/100
w4 = w0/500
W = [w1, 4.3*w1, 4.7*w1, w4]

In [227]:
def t(NT): # Time
    t = np.linspace(0, NT/fn, NT)
    return t 

def CN(NT):
    CN = np.random.normal(0, 400*0.5, int(NT))
    return CN

def s(t): # The summation
    s = 0
    for i in range(len(A)-1):
        s += A[i] * cos(W[i]*t) 
    return s
                        
def S(NT): # Array of summation values
    S = s(t(NT))
    return S  

def x(NT): # Array of x(t) values
    x = CN(NT) + S(NT)
    return x

In [237]:
NT = [20000, 2000]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
ax1.plot(t(NT[0]), x(NT[0]))
ax2.plot(t(NT[1]), x(NT[1]))
ax1.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 20000 sampled at 10Hz', fontsize=14)
ax2.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 2000 sampled at 10Hz', fontsize=14)
ax1.set_xlabel('Time ($s$)', fontsize=12)
ax2.set_xlabel('Time ($s$)', fontsize=12)

fig.tight_layout()

<Figure size 864x720 with 2 Axes>

### Step-by-step calculation of PSD:

#### For NT = 20000

#### For NT = 2000

#### Questions: 

##### 1) Does increasing the record length improve the estimate? 



##### 2) How does the record length affect the frequency resolution?

### Use built in function of pwelch (well...the Python equivalent):